# Function calling

https://ai.google.dev/docs/function_calling

In [2]:
import os
GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']

In [79]:
def user(text):
    return { 
        'role': 'user',
        'parts': [{
            'text': text
        }]
    }

def function_response(name, content):
    return {
        "role": "function",
        "parts": [{
          "functionResponse": {
            "name": name,
            "response": {
              "name": name,
              "content": content
            }
          }
        }]
      }

def tools():
    return [
        {
            'name': 'datetime_now',
            'description': 'returns the current date'
        },
        {
            'name': 'web_search',
            'description': 'browse the web for content',
            'parameters': {
                'type': 'object',
                'properties': {
                    'query': {
                        'type': 'string',
                        'description': 'the search keyword'
                    }
                },
                'required': ['query']
            }
        }
    ]

In [80]:
import requests

params = {
    'key': GOOGLE_API_KEY
}
data = {
    'contents': [user("what is today's date?")],
    'tools': [
        {'function_declarations': tools()}
    ]
}
print(data)

{'contents': [{'role': 'user', 'parts': [{'text': "what is today's date?"}]}], 'tools': [{'function_declarations': [{'name': 'datetime_now', 'description': 'returns the current date'}, {'name': 'web_search', 'description': 'browse the web for content', 'parameters': {'type': 'object', 'properties': {'query': {'type': 'string', 'description': 'the search keyword'}}, 'required': ['query']}}]}]}


In [81]:
response = requests.post("https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent", params=params, json=data)
response

<Response [200]>

In [82]:
print(response.json())

{'candidates': [{'content': {'parts': [{'functionCall': {'name': 'datetime_now', 'args': {}}}], 'role': 'model'}, 'finishReason': 'STOP', 'index': 0}], 'promptFeedback': {'safetyRatings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE'}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE'}]}}


In [89]:
from datetime import datetime

function_call = response.json()['candidates'][0]['content']
function_resp = function_response(function_call['parts'][0]['functionCall']['name'], str(datetime.now()))

In [90]:
import json

params = {
    'key': GOOGLE_API_KEY
}
data = {
    'contents': [user("what is today's date?"), function_call, function_resp],
    'tools': [
        {'function_declarations': tools()}
    ]
}
print(json.dumps(data, indent=2))

{
  "contents": [
    {
      "role": "user",
      "parts": [
        {
          "text": "what is today's date?"
        }
      ]
    },
    {
      "parts": [
        {
          "functionCall": {
            "name": "datetime_now",
            "args": {}
          }
        }
      ],
      "role": "model"
    },
    {
      "role": "function",
      "parts": [
        {
          "functionResponse": {
            "name": "datetime_now",
            "response": {
              "name": "datetime_now",
              "content": "2023-12-25 02:48:05.807683"
            }
          }
        }
      ]
    }
  ],
  "tools": [
    {
      "function_declarations": [
        {
          "name": "datetime_now",
          "description": "returns the current date"
        },
        {
          "name": "web_search",
          "description": "browse the web for content",
          "parameters": {
            "type": "object",
            "properties": {
              "query": {
               

In [91]:
tools_response = requests.post("https://generativelanguage.googleapis.com/v1beta/models/gemini-pro:generateContent", params=params, json=data)
tools_response

<Response [200]>

In [92]:
print(tools_response.text)

{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": " Today is December 25th, 2023."
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",
      "index": 0,
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE"
        }
      ]
    }
  ]
}

